# --- 1. Importações e Configurações ---


In [ ]:
!pip install gensim thefuzz

In [ ]:
# Manipulação de dados
import pandas as pd
import numpy as np
import altair as alt

# Pré-processamento de texto
import re
import nltk
from nltk.corpus import stopwords

# Vetorização
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer, HashingVectorizer
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.decomposition import PCA

# Modelagem (Clustering)
from sklearn.cluster import KMeans, AgglomerativeClustering
from sklearn.pipeline import make_pipeline

# Avaliação
from sklearn.metrics import silhouette_score, pairwise_distances

# Para Word Embeddings (opcional, mas recomendado)
from gensim.models import Word2Vec

# Garantir que os pacotes do NLTK estão baixados
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('punkt_tab')


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


True

# --- 2. Carregamento e Pré-processamento ---


## União

In [ ]:
# Carregando os dados
df1 = pd.read_csv('https://raw.githubusercontent.com/astromar2187/CienciadeDados1/refs/heads/main/df_final.csv')
# Visualizando informações básicas
print(df1.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3226 entries, 0 to 3225
Data columns (total 9 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   artist        3226 non-null   object
 1   title         3226 non-null   object
 2   album         3226 non-null   object
 3   year          3226 non-null   int64 
 4   lyrics        3210 non-null   object
 5   tags          109 non-null    object
 6   record_label  48 non-null     object
 7   estado        3226 non-null   object
 8   regiao        3226 non-null   object
dtypes: int64(1), object(8)
memory usage: 227.0+ KB
None


In [ ]:
df2 = pd.read_csv('https://raw.githubusercontent.com/astromar2187/CienciadeDados1/refs/heads/main/all_artists.csv')

print(df2.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2158 entries, 0 to 2157
Data columns (total 7 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   artist        2158 non-null   object
 1   tags          2158 non-null   object
 2   album         2158 non-null   object
 3   year          2158 non-null   int64 
 4   record_label  941 non-null    object
 5   title         2158 non-null   object
 6   lyrics        1680 non-null   object
dtypes: int64(1), object(6)
memory usage: 118.1+ KB
None


In [ ]:
# Criar um dicionário mapeando 'artist_id' do df2 para 'tags'
# Assumimos que df2 tem colunas 'artist_id' e 'tags'
artist_tags_map = df2.set_index('artist')['tags'].to_dict()

df1['tags'] = df1['artist'].map(artist_tags_map)

df1['tags'].fillna('', inplace=True) # Ou None, dependendo do que você preferir

print(df1.head())
print(df1.info())
df1.to_csv('df_final.csv', index=False)

             artist                             title  album  year  \
0  akira presidente                 assumindo o risco  nandi  2019   
1  akira presidente                   vivo como quero  nandi  2019   
2  akira presidente                             livre  nandi  2019   
3  akira presidente                             dance  nandi  2019   
4  akira presidente  ela sobe, ela desce (part. aina)  nandi  2019   

                                              lyrics tags record_label  \
0  ['meu', 'vitoria', 'nao', 'ser', 'sorte', 'fac...               NaN   
1  ['fazer', 'o', 'que', 'eu', 'gostar', 'eu', 'v...               NaN   
2  ['Akira', 'presidente', 'livre', 'de', 'negati...               NaN   
3  ['dance', 'como', 'se', 'nada', 'importar', 'o...               NaN   
4  ['ele', 'subir', 'ele', 'descer', 'ele', 'joga...               NaN   

           estado   regiao  
0  Rio de Janeiro  Sudeste  
1  Rio de Janeiro  Sudeste  
2  Rio de Janeiro  Sudeste  
3  Rio de Janeiro 

/tmp/ipython-input-133-550478443.py:7: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df1['tags'].fillna('', inplace=True) # Ou None, dependendo do que você preferir


In [ ]:
df1['tags']

,tags
0,
1,
2,
3,
4,
...,...
3221,
3222,
3223,
3224,


In [ ]:
df = df1

## Contagem e limpeza

In [ ]:
df['titulo_limpo'] = df['title'].str.normalize('NFKD').str.encode('ascii', errors='ignore').str.decode('utf-8')
df.drop_duplicates(subset ="titulo_limpo",
                       keep = 'first', inplace = True)

song_count_by_artist = df.groupby('artist')['title'].nunique().sort_values(ascending=False)
song_count_by_artist.reset_index(name='song_count')

,artist,song_count
0,marcelo d2,122
1,emicida,120
2,projota,119
3,tribodaperiferia,105
4,gabrielpensador,102
...,...,...
73,pollo,2
74,tashaetracie,2
75,hungria,2
76,mc marcinho,2


In [ ]:
try:
    portuguese_stopwords = set(stopwords.words("portuguese"))
except LookupError:
    print("Baixando a lista de stopwords do NLTK para o português...")
    nltk.download('stopwords')
    portuguese_stopwords = set(stopwords.words("portuguese"))

# Adiciona suas palavras customizadas à lista principal
custom_stopwords = {
    'que', 'até', 'esse', 'essa', 'pro', 'pra', 'oi', 'lá', 'blá', 'nan',
    'bb', 'bbm', 'abm', 'cbm', 'dbm', 'dos', 'ltda', 'editora', 'é', 'tá'
}
stop_words = portuguese_stopwords.union(custom_stopwords)
stop_words

{'a',
 'abm',
 'ao',
 'aos',
 'aquela',
 'aquelas',
 'aquele',
 'aqueles',
 'aquilo',
 'as',
 'até',
 'bb',
 'bbm',
 'blá',
 'cbm',
 'com',
 'como',
 'da',
 'das',
 'dbm',
 'de',
 'dela',
 'delas',
 'dele',
 'deles',
 'depois',
 'do',
 'dos',
 'e',
 'editora',
 'ela',
 'elas',
 'ele',
 'eles',
 'em',
 'entre',
 'era',
 'eram',
 'essa',
 'essas',
 'esse',
 'esses',
 'esta',
 'estamos',
 'estar',
 'estas',
 'estava',
 'estavam',
 'este',
 'esteja',
 'estejam',
 'estejamos',
 'estes',
 'esteve',
 'estive',
 'estivemos',
 'estiver',
 'estivera',
 'estiveram',
 'estiverem',
 'estivermos',
 'estivesse',
 'estivessem',
 'estivéramos',
 'estivéssemos',
 'estou',
 'está',
 'estávamos',
 'estão',
 'eu',
 'foi',
 'fomos',
 'for',
 'fora',
 'foram',
 'forem',
 'formos',
 'fosse',
 'fossem',
 'fui',
 'fôramos',
 'fôssemos',
 'haja',
 'hajam',
 'hajamos',
 'havemos',
 'haver',
 'hei',
 'houve',
 'houvemos',
 'houver',
 'houvera',
 'houveram',
 'houverei',
 'houverem',
 'houveremos',
 'houveria',
 'h

In [ ]:
def limpar_letra(letra):
    """
    Função para limpar e pré-processar o texto de uma letra de música.
    1. Verifica se o texto é uma string válida.
    2. Converte para minúsculas.
    3. Remove caracteres especiais, números e pontuações.
    4. Remove palavras com 2 ou menos caracteres.
    5. Remove as stopwords.
    """
    # Retorna uma string vazia se a entrada não for um texto (ex: NaN)
    if not isinstance(letra, str):
        return ""

    # Converte para minúsculas
    letra = letra.lower()

    # Remove tudo que não for letra (números, pontuações, etc.)
    letra = re.sub(r'[^a-z\s]', '', letra)

    # Cria uma lista de palavras, removendo as stopwords e palavras curtas
    palavras_limpas = [
        palavra
        for palavra in letra.split()
        if palavra not in stop_words and len(palavra) > 2
    ]

    # Junta as palavras limpas em uma única string
    return ' '.join(palavras_limpas)

In [ ]:
df['clean_lyrics'] = df['lyrics'].apply(limpar_letra)

print(df['clean_lyrics'].head().tolist())

['vitoria nao sorte facil achar heranca acidente grana faco vir bala abaixo saber bom gente medo morrer viver vao medo morrer viver vao medo morrer viver vao deus protejer enquanto voltar missao assumir risco feliz rico falar sacrificio vida andar quinto disco rimar toa desperdicio saudade inicio pouco grana quase tudo dificil querer sair merda bem forte vicio rastejar nao opcao ferido pior caminhada sofrer nao ter pior yeezy falso altura falso sorriso amor amer tudo preciso estrada longo aproveitar viagem destino distante aqui avisar casa chegar tarde voltar ver sorrir vitoria nao sorte facil achar heranca acidente grana faco vir bala abaixo saber bom gente medo morrer viver vao medo morrer viver vao medo morrer viver vao deus protejer enquanto voltar missao raiva mover correr errar pouco bolso pouco lado nao morri baixa abencoar pior ter parar lembrar ter fugir silencio premio ficar esquecido rap frio sentido milhao sonho vario coracoes partido foda parar ouvir voz mente tentar atras

# --- 3. Central de Vetorização ---


In [ ]:
class Word2VecVectorizer(BaseEstimator, TransformerMixin):
    def __init__(self, vector_size=100, window=5, min_count=3, workers=4):
        self.vector_size = vector_size
        self.window = window
        self.min_count = min_count
        self.workers = workers
        self.model = None
        self.word_vectors = None

    def fit(self, X, y=None):
        # X é uma lista de strings (clean_lyrics)
        tokenized_lyrics = [nltk.word_tokenize(letra) for letra in X]
        self.model = Word2Vec(
            sentences=tokenized_lyrics,
            vector_size=self.vector_size,
            window=self.window,
            min_count=self.min_count,
            workers=self.workers
        )
        self.word_vectors = self.model.wv # Armazenar keydvectors para acesso rapido
        return self

    def transform(self, X):
        # X é uma lista de strings (clean_lyrics)
        tokenized_lyrics = [nltk.word_tokenize(letra) for letra in X]

        def vectorize_document(doc, model):
            word_vectors = [model[word] for word in doc if word in model]
            if not word_vectors:
                return np.zeros(self.vector_size)
            return np.mean(word_vectors, axis=0)

        # Criar a matriz de features
        X_vec = np.array([vectorize_document(doc, self.word_vectors) for doc in tokenized_lyrics])
        return X_vec


In [ ]:
# Dicionário com os vetorizadores que vamos testar
vectorizers = {
    "TfidfVectorizer": TfidfVectorizer(
        max_features=1000, # Limita o vocabulário para as 1000 palavras mais comuns
        max_df=0.8,       # Ignora palavras que aparecem em mais de 80% das letras
        min_df=5,         # Ignora palavras que aparecem em menos de 5 letras
        ngram_range=(1,2) # Considera palavras isoladas (unigramas) e pares de palavras (bigramas)
    ),
    "CountVectorizer": CountVectorizer(
        max_features=1000,
        max_df=0.8,
        min_df=5,
        ngram_range=(1,1) # Apenas unigramas para o baseline
    ),
    "HashingVectorizer": HashingVectorizer(
        n_features=2**10, # Potência de 2, ex: 1024 features
        ngram_range=(1,2),
        norm='l2' # Normaliza os vetores, similar ao TF-IDF
    ),
    "Word2Vec": Word2VecVectorizer(
        vector_size=100,
        window=5,
        min_count=3,
        workers=4)
}

print(f"Vetorizadores prontos para teste: {list(vectorizers.keys())}")

Vetorizadores prontos para teste: ['TfidfVectorizer', 'CountVectorizer', 'HashingVectorizer', 'Word2Vec']


### --- Vetorizações ---

In [ ]:
tfidf_vec_text = vectorizers["TfidfVectorizer"].fit_transform(df['clean_lyrics'])
count_vec_text = vectorizers["CountVectorizer"].fit_transform(df['clean_lyrics'])
hash_vec_text = vectorizers["HashingVectorizer"].fit_transform(df['clean_lyrics'])
w2v_vec_text = vectorizers["Word2Vec"].fit_transform(df['clean_lyrics'])

In [ ]:
vetores = {
    "TfidfVectorizer": tfidf_vec_text,
    "CountVectorizer": count_vec_text,
    "HashingVectorizer": hash_vec_text,
    "Word2Vec": w2v_vec_text
}

# --- 4: Decidindo o Número Ideal de Clusters (K) ---


In [ ]:
for nome_vetorizador, vec_text in vetores.items():

    print(f"\n--- Iniciando análise para o vetorizador: {nome_vetorizador} ---")

    # Define o intervalo de K que vamos testar.
    # Começamos com k=2 porque o Coeficiente de Silhueta não pode ser calculado para k=1.
    k_values = range(2, 16)
    ssd_scores_kmeans = []
    silhouette_scores_kmeans = []
    silhouette_scores_agglo = [] # Nova lista para Agglomerative Clustering

    print(f"Testando K de {min(k_values)} a {max(k_values)} com KMeans padrão...")

    for k in k_values:
        # Instancia o KMeans com os parâmetros mais comuns
        kmeans = KMeans(
            n_clusters=k,
            init='k-means++',   # Método de inicialização inteligente
            random_state=42     # Garante que os resultados sejam os mesmos em cada execução
        )

        # Treina o modelo com os dados vetorizados da iteração atual
        kmeans.fit(vec_text)

        # Armazena as métricas
        ssd_scores_kmeans.append(kmeans.inertia_)
        silhouette_scores_kmeans.append(silhouette_score(vec_text, kmeans.labels_))

        # AgglomerativeClustering requires dense data. Convert sparse matrices if necessary.
        if hasattr(vec_text, "toarray"):
            vec_text_dense = vec_text.toarray()
        else:
            vec_text_dense = vec_text

        agglo = AgglomerativeClustering(
            n_clusters=k,
            metric='cosine', # 'affinity' foi renomeado para 'metric' no scikit-learn > 0.24
            linkage='average' # 'average' ou 'complete' são boas escolhas para 'cosine'
        )
        labels_agglo = agglo.fit_predict(vec_text_dense) # Use the dense data here
        silhouette_scores_agglo.append(silhouette_score(vec_text_dense, labels_agglo)) # Use the dense data here


    # Cria um DataFrame único com todos os resultados para o vetorizador atual
    qualidade_df = pd.DataFrame({
        'k': k_values,
        'ssd_kmeans': ssd_scores_kmeans,
        'silhouette_kmeans': silhouette_scores_kmeans,
        'silhouette_agglo': silhouette_scores_agglo # Adicionado
    })

    print(f"Cálculo das métricas para '{nome_vetorizador}' concluído.")

    # --- Gráfico 1: Método do Cotovelo (SSD) para KMeans ---
    elbow_plot = alt.Chart(qualidade_df).mark_line(point=True).encode(
        x=alt.X('k:Q', title='Número de Clusters (k)', axis=alt.Axis(tickMinStep=1)),
        y=alt.Y('ssd_kmeans:Q', title='Soma dos Quadrados das Distâncias (SSD)', scale=alt.Scale(zero=False))
    ).properties(
        title=f'Gráfico de Cotovelo (KMeans - {nome_vetorizador})'
    )

    # --- Gráfico 2: Coeficiente de Silhueta para KMeans e AgglomerativeClustering ---
    # Combine os dados de silhueta para plotar no mesmo gráfico
    silhouette_data = qualidade_df.melt('k', value_vars=['silhouette_kmeans', 'silhouette_agglo'],
                                       var_name='Algoritmo', value_name='Coeficiente de Silhueta')

    silhouette_plot = alt.Chart(silhouette_data).mark_line(point=True).encode(
        x=alt.X('k:Q', title='Número de Clusters (k)', axis=alt.Axis(tickMinStep=1)),
        y=alt.Y('Coeficiente de Silhueta:Q', title='Coeficiente de Silhueta Médio'),
        color='Algoritmo:N' # Diferencia as linhas por algoritmo
    ).properties(
        title=f'Análise de Silhueta ({nome_vetorizador})'
    )

    display(elbow_plot | silhouette_plot)


--- Iniciando análise para o vetorizador: TfidfVectorizer ---
Testando K de 2 a 15 com KMeans padrão...


ValueError: Cosine affinity cannot be used when X contains zero vectors

# --- 5. Modelagem e Avaliação em Loop ---


In [ ]:
# Parâmetros do K-Means (mantenha fixo para uma comparação justa)
N_CLUSTERS = 10
kmeans_model = KMeans(
    n_clusters=N_CLUSTERS,
    init='k-means++',
    max_iter=300,
    random_state=42 # Para reprodutibilidade
)

# Dicionário para armazenar os resultados
results = {}

print("Iniciando o treinamento e avaliação dos modelos...")
print("-" * 30)

for name, vectorizer in vectorizers.items():
    print(f"Testando o vetorizador: {name}")

    # Cria um pipeline: 1. Vetoriza, 2. Aplica o K-Means
    pipeline = make_pipeline(vectorizer, kmeans_model)

    # Treina o modelo e obtém os clusters
    labels = pipeline.fit_predict(df['clean_lyrics'])

    # Pega a matriz de features gerada pelo vetorizador para calcular a silhueta
    # (O pipeline.steps[0][1] acessa o vetorizador treinado)
    X_features = pipeline.steps[0][1].fit_transform(df['clean_lyrics'])

    # Calcula o Coeficiente de Silhueta
    # (Ignora o cálculo se a matriz for muito esparsa e der erro, como pode acontecer com Hashing)
    try:
        silhouette = silhouette_score(X_features, labels)
        print(f"  -> Coeficiente de Silhueta: {silhouette:.4f}")
    except Exception as e:
        silhouette = -1 # Valor para indicar erro
        print(f"  -> Não foi possível calcular a silhueta: {e}")

    # Armazena o resultado
    results[name] = {
        'silhouette_score': silhouette,
        'pipeline': pipeline # Guarda o modelo treinado para análise posterior
    }
    print("-" * 30)

print("Avaliação concluída")

Iniciando o treinamento e avaliação dos modelos...
------------------------------
Testando o vetorizador: TfidfVectorizer
  -> Coeficiente de Silhueta: -0.0369
------------------------------
Testando o vetorizador: CountVectorizer
  -> Coeficiente de Silhueta: 0.0377
------------------------------
Testando o vetorizador: HashingVectorizer
  -> Coeficiente de Silhueta: -0.0422
------------------------------
Testando o vetorizador: Word2Vec
  -> Coeficiente de Silhueta: 0.0845
------------------------------
Avaliação concluída


# --- 6. Análise Comparativa dos Resultados ---

### --- Selecionar o Melhor Modelo e Adicionar os Labels ao DataFrame ---

In [ ]:
# Converte o dicionário de resultados em um DataFrame para fácil visualização
results_df = pd.DataFrame.from_dict(results, orient='index').sort_values(
    by='silhouette_score',
    ascending=False
)
# Pega o nome do melhor vetorizador do ranking que criamos anteriormente
best_vectorizer_name = results_df.index[0]
best_pipeline = results[best_vectorizer_name]['pipeline']

print(f"Analisando os resultados do melhor modelo: '{best_vectorizer_name}'")

Analisando os resultados do melhor modelo: 'Word2Vec'


In [ ]:
results[best_vectorizer_name]['pipeline']

Pipeline(steps=[('word2vecvectorizer', Word2VecVectorizer()),
                ('kmeans', KMeans(n_clusters=10, random_state=42))])

In [ ]:
# Adiciona uma nova coluna 'cluster' ao DataFrame original com os resultados do melhor modelo.
# É importante usar o DataFrame completo (df_final) para a predição, mas garantir
# que o texto usado seja o mesmo que foi usado no treinamento (df['clean_lyrics']).
# Primeiro, pegamos os índices de `df['clean_lyrics']` para alinhar corretamente.
df_com_clusters = df.loc[df['clean_lyrics'].index].copy()
df_com_clusters['cluster'] = best_pipeline.predict(df['clean_lyrics'])

### --- Tamanho dos Clusters ---

In [ ]:
print("\n--- 1. Análise de Tamanho dos Clusters ---")
cluster_sizes = df_com_clusters['cluster'].value_counts().sort_index()
print("Número de músicas em cada cluster:")
display(cluster_sizes)


--- 1. Análise de Tamanho dos Clusters ---
Número de músicas em cada cluster:


,count
cluster,
0,521
1,281
2,286
3,17
4,70
5,326
6,692
7,9
8,1


### --- Palavras-Chave por Cluster ---

In [ ]:
print("\n--- 2. Análise de Palavras-Chave (Temas) por Cluster e Amostra de Músicas ---")

# Extrai o vetorizador e o modelo KMeans do pipeline
cluster_keywords_map = {}
vectorizer = vectorizers[best_vectorizer_name]
kmeans = best_pipeline.named_steps['kmeans']

if isinstance(vectorizer, Word2VecVectorizer):
    for i in range(kmeans.n_clusters):
        centroid_vector = kmeans.cluster_centers_[i]
        most_similar_words = vectorizer.model.wv.most_similar(positive=[centroid_vector], topn=10)
        cluster_keywords_map[i] = [word for word, sim in most_similar_words]
else:
    order_centroids = kmeans.cluster_centers_.argsort()[:, ::-1]
    terms = vectorizer.get_feature_names_out()
    for i in range(kmeans.n_clusters):
        cluster_keywords_map[i] = [terms[ind] for ind in order_centroids[i, :10]]

# Agora, iteramos para mostrar os temas e as amostras de músicas
for i in range(kmeans.n_clusters):
    # Pega os 3 primeiros temas do nosso mapa de palavras-chave
    tema_cluster = ' / '.join(cluster_keywords_map[i][:3])

    print(f"\n===== Cluster {i} (Tema: {tema_cluster}) =====")

    # Pega o tamanho do cluster para a amostragem segura
    cluster_size = df_com_clusters[df_com_clusters['cluster'] == i].shape[0]

    sample_songs = df_com_clusters[df_com_clusters['cluster'] == i].sample(
        n=min(5, cluster_size), # Garante que não tentamos amostrar mais músicas do que existem
        random_state=42
    )
    for _, row in sample_songs.iterrows():
        print(f"  - '{row['title']}' por '{row['artist']}'")


--- 2. Análise de Palavras-Chave (Temas) por Cluster e Amostra de Músicas ---

===== Cluster 0 (Tema: desandar / brincadeira / fudido) =====
  - 'tem que ter atitude' por 'tribo-da-periferia'
  - 'a rima denuncia' por 'gog'
  - 'my lovers' por 'costa gold'
  - 'zerovinteum' por 'planet hemp'
  - 'a lei de gerson' por 'gog'

===== Cluster 1 (Tema: entaar / bis / interessar) =====
  - 'marcha' por 'febem'
  - 'fogo no teto' por 'mc hariel sp'
  - 'sensacao (part. lelle)' por 'rincon-sapiencia'
  - 'o sopro da cigarra' por 'gabriel-pensador'
  - 'agua' por 'sidoka'

===== Cluster 2 (Tema: iludir / esqueca / bastar) =====
  - 'vivencias' por 'sant'
  - 'danca do desempregado' por 'gabriel-pensador'
  - 'sem chances' por '509-e'
  - 'lenda (part. qtz tivityn)' por 'raffa moreira'
  - 'nao me espere' por 'bk-nectar'

===== Cluster 3 (Tema: vic / sinistrar / cia) =====
  - 'afropunk no valle do rap' por 'marcelo d2'
  - 'gorilla grip' por 'planet hemp'
  - 'Mó Blef' por 'Apocalipse 16'
  - '

### --- Visualização dos Clusters com PCA ---

In [ ]:
print("\n--- 4. Visualização dos Clusters em 2D ---")

# Em vez de chamar fit_transform novamente, apenas transformamos os dados com o pipeline já treinado.
# O pipeline cuida de chamar o método .transform correto para cada passo.
X_features = best_pipeline.transform(df['clean_lyrics'])

# Reduz a dimensionalidade para 2D usando PCA.
# Adicionamos uma verificação para evitar o erro com .toarray().
pca = PCA(n_components=2, random_state=42)

# Se X_features for uma matriz esparsa (de TF-IDF, etc.), converta para densa.
# Se já for densa (de Word2Vec), use diretamente.
if hasattr(X_features, "toarray"):
    print("Matriz esparsa detectada. Convertendo para densa antes do PCA.")
    X_pca = pca.fit_transform(X_features.toarray())
else:
    print("Matriz densa (Word2Vec) detectada. Usando diretamente para o PCA.")
    X_pca = pca.fit_transform(X_features)

# Cria um DataFrame para a visualização com Altair (sem alterações aqui)
df_pca = pd.DataFrame({
    'pca1': X_pca[:, 0],
    'pca2': X_pca[:, 1],
    'cluster': df_com_clusters['cluster'],
    'title': df_com_clusters['title'],
    'artist': df_com_clusters['artist']
})

# Cria o gráfico de dispersão
chart = alt.Chart(df_pca).mark_circle(size=60).encode(
    x=alt.X('pca1', title='Componente Principal 1'),
    y=alt.Y('pca2', title='Componente Principal 2'),
    color=alt.Color('cluster:N', title='Cluster', scale=alt.Scale(scheme='category10')),
    tooltip=['title', 'artist', 'cluster']
).properties(
    title=f'Visualização de Clusters (Modelo: {best_vectorizer_name})',
    width=700,
    height=500
).interactive()

# Exibe o gráfico
display(chart)


--- 4. Visualização dos Clusters em 2D ---
Matriz densa (Word2Vec) detectada. Usando diretamente para o PCA.


alt.Chart(...)

In [ ]:
results

{'TfidfVectorizer': {'silhouette_score': -0.036873820470911776,
  'pipeline': Pipeline(steps=[('tfidfvectorizer',
                   TfidfVectorizer(max_df=0.8, max_features=1000, min_df=5,
                                   ngram_range=(1, 2))),
                  ('kmeans', KMeans(n_clusters=10, random_state=42))])},
 'CountVectorizer': {'silhouette_score': 0.03769916359333098,
  'pipeline': Pipeline(steps=[('countvectorizer',
                   CountVectorizer(max_df=0.8, max_features=1000, min_df=5)),
                  ('kmeans', KMeans(n_clusters=10, random_state=42))])},
 'HashingVectorizer': {'silhouette_score': -0.042158181867219806,
  'pipeline': Pipeline(steps=[('hashingvectorizer',
                   HashingVectorizer(n_features=1024, ngram_range=(1, 2))),
                  ('kmeans', KMeans(n_clusters=10, random_state=42))])},
 'Word2Vec': {'silhouette_score': 0.08446344666855443,
  'pipeline': Pipeline(steps=[('word2vecvectorizer', Word2VecVectorizer()),
                  (